In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from DeepPurpose import utils, models, dataset

In [3]:
import numpy as np
import pandas as pd
import json
import os 
print(os.path.abspath('.'))

/root/data/DeepPurpose


In [4]:
Smile = np.load("DB_smiles.npy", allow_pickle=True)
Target = np.load("DB_targets.npy", allow_pickle=True)
y = np.load("DB_y.npy", allow_pickle=True)
print('Done!')

Done!


In [5]:
X_drugs, X_targets, y = Smile, Target, y
print('Drug 1: ' + X_drugs[0])
print('Target 1: ' + X_targets[0])
print('Score 1: ' + str(y[0]))

Drug 1: Cc1ccc(CNS(=O)(=O)c2ccc(s2)S(N)(=O)=O)cc1
Target 1: MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPLSVSYDQATSLRILNNGHAFNVEFDDSQDKAVLKGGPLDGTYRLIQFHFHWGSLDGQGSEHTVDKKKYAAELHLVHWNTKYGDFGKAVQQPDGLAVLGIFLKVGSAKPGLQKVVDVLDSIKTKGKSADFTNFDPRGLLPESLDYWTYPGSLTTPPLLECVTWIVLKEPISVSSEQVLKFRKLNFNGEGEPEELMVDNWRPAQPLKNRQIKASFK
Score 1: 9.251811972993801


In [6]:
drug_encoding, target_encoding = 'CNN', 'AAC'

In [7]:
train, val, test = utils.data_process(X_drugs, X_targets, y, 
                                drug_encoding, target_encoding, 
                                split_method='train_full',frac=[0.8,0.1,0.1],
                                random_seed = 1)
train.head(1)

in total: 66434 drug-target pairs
encoding drug...
unique drugs: 10661
drug encoding finished...
encoding protein...
unique target sequence: 1413
-- Encoding AAC takes time. Time Reference: 24s for ~100 sequences in a CPU.					 Calculate your time by the unique target sequence #, instead of the entire dataset.
protein encoding finished...
splitting dataset...
Done.


,SMILES,Target Sequence,Label,drug_encoding,target_encoding
0,Cc1ccc(CNS(=O)(=O)c2ccc(s2)S(N)(=O)=O)cc1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,9.251812,"[C, c, 1, c, c, c, (, C, N, S, (, =, O, ), (, ...","[5.0, 2.692, 3.846, 7.308, 0.385, 5.0, 4.231, ..."


In [8]:
config = utils.generate_config(drug_encoding = drug_encoding, 
                         target_encoding = target_encoding, 
                         cls_hidden_dims = [1024,1024,512], 
                         train_epoch = 200, 
                         test_every_X_epoch = 10, 
                         LR = 0.001, 
                         #decay = 0.001,
                         batch_size = 1024,
                         #hidden_dim_drug = 128,
                         #mpnn_hidden_size = 128,
                         #mpnn_depth = 4, 
                         #cnn_target_filters = [32,64,96],
                         #cnn_target_kernels = [4,8,12],
                         cnn_drug_filters = [32,64,96],
                         cnn_drug_kernels = [4,6,8], 
                         num_workers = 10
                              )

In [ ]:
model = models.model_initialize(**config)
model.config

In [ ]:
model.train(train, val, test)

In [10]:
model.save_model('./model_CNN_AAC_DB')